<a href="https://colab.research.google.com/github/Anastasiamay/Study_projects/blob/main/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ базы данных сервиса для чтения книг по подписке

База данных представляет собой несколько таблиц, которые включают информацию о книгах, издательствах, авторах, а также пользовательских обзорах и оценках книг.
Задачи исследования: 
* исследовать таблицы
* определить количество книг, вышедших после 1 января 2000 года
* определить количество обзоров и среднюю оценку для каждой книги
* определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключить из анализа брошюры)
* определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)
* определить среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings("ignore")

In [15]:
conn = psycopg2.connect(
    host="********",
    port=****,
    database="*********",
    user='*******',
    password="**********",
)
cur = conn.cursor()
cur.execute("SELECT * FROM books LIMIT 5")
data = cur.fetchall()

In [16]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table in tables:
  df = pd.read_sql(f'SELECT * FROM {table} LIMIT 5', conn)
  print("Таблица", table)
  display(df)
  print()

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `books` содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

Таблица `ratings` содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Количество книг, вышедших после 1 января 2000 года

In [17]:
pd.read_sql('''
SELECT count(*) 
FROM books 
WHERE publication_date > '2000-01-01'
''', conn)

,count
0,819


В базе данных сервиса для чтения находится 819 книг, вышедших после 1 января 2000 года

Количество обзоров и средняя оценка для каждой книги

In [18]:
pd.read_sql('''
SELECT books.book_id, title, count(distinct review_id) as cnt_review, round(avg(rating), 2) as avg_rating
FROM books 
LEFT JOIN reviews on books.book_id =  reviews.book_id
LEFT JOIN ratings on books.book_id = ratings.book_id
GROUP BY books.book_id, title
ORDER BY cnt_review desc, avg_rating DESC
''', conn)

,book_id,title,cnt_review,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Всего в базе данных 1000 книг. На некоторые книги отсутствуют обзоры, максимальное количество обзоров на одну книгу - 7

Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем брошюры) 

In [19]:
pd.read_sql('''
SELECT publisher, cnt 
FROM(
    SELECT publisher, count(book_id) as cnt
    FROM publishers
    JOIN books on books.publisher_id =  publishers.publisher_id
    WHERE num_pages in (
                        SELECT num_pages 
                        FROM books 
                        WHERE num_pages > 50)
    GROUP BY publisher
    ORDER BY count(book_id) DESC) t
LIMIT 1
''', conn)

,publisher,cnt
0,Penguin Books,42


Наибольшее количество книг - 42, выпустило издательство Penguin Books, исключая брошюры (количество страниц более 50)

Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [20]:
pd.read_sql('''
SELECT author, avg_rating 
FROM(
    SELECT author, round(avg(rating), 2) as avg_rating
    FROM authors
    JOIN books on books.author_id = authors.author_id
    JOIN ratings on ratings.book_id = books.book_id
    WHERE books.book_id in (
                            SELECT book_id 
                            FROM ratings 
                            GROUP BY book_id 
                            HAVING count(rating_id) >= 50)
    GROUP BY author
    ORDER BY avg_rating desc) t
    LIMIT 1
''', conn)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Среди книг с 50 и более оценками, самая высокая средняя оценка ~4.3 у автора J.K. Rowling в сотрудничестве с иллюстратором Mary GrandPré

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
pd.read_sql('''
SELECT round(avg(cnt),1) as avg_cnt_review
FROM(
    SELECT username, count(review_id) as cnt
    FROM reviews
    WHERE username in (
                      SELECT username
                      FROM ratings 
                      GROUP BY username
                      HAVING count(rating_id) > 50)
    GROUP BY username) t                            
''', conn)

,avg_cnt_review
0,24.3


В среднем, количество обзоров от пользователей, которые поставили более 50 оценок составляет 24 обзора

# Выводы
* Количество книг в базе данных 1000 книг
* Количество книг, вышедших после 1 января 2000 года - 819
* На некоторые книги отсутствуют обзоры, максимальное количество обзоров на одну книгу - 7
* Наибольшее количество книг - 42, выпустило издательство Penguin Books, исключая брошюры (количество страниц более 50)
* Среди книг с 50 и более оценками, самая высокая средняя оценка ~4.3 у автора J.K. Rowling в сотрудничестве с иллюстратором Mary GrandPré
* Среднее количество обзоров от пользователей, которые поставили более 50 оценок - 24